In [129]:
#Import the needed modules
import klayout as kl
import klayout.db as db
import pya
import pandas as pd

#Define the function
def rpdb(gds_file:str = 'Contact_test_tD_s100_p200_l10000_a0_3to1.gds', mesh:str = 'Test_Mesh.csv', target_layer:int = 2, LL:list=(-5,-5), UR:list=(5,5)):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = pya.Layout()
    gds.read(gds_file)

    
    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Determine number of layers for copy and bias layer indicies (based on order of creation in script)
    num_lay = len(gds.layer_indexes())
    copy_lyr_index = num_lay
    bias_lyr_index = num_lay+1

    #Create copy layer, copy structures over from the desired layer for biasing.
    #This copy layer will be mauled by the bias layer so that the original target layer is untouched.
    copy_lyr = gds.layer(97,0)
    gds.copy_layer(target_layer,copy_lyr_index)

    #Defines a bounding box for the copy layer
    global_bbox = gds.top_cell().dbbox_per_layer(copy_lyr)


    #Create bias layer for eventual use
    bias_lyr = gds.layer(98,0)

    #Create final bias layer
    final_lyr = gds.layer(99,0)

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    
    #Determine mesh columns and convert to string for DataFrame access
    col_len = len(pdb.columns)+1
    print(col_len)
    columns = range(1,col_len)
    columns_str = [str(x) for x in columns]

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))

    top_cell = gds.top_cell()
    bias_cell_top = gds.create_cell('bias_cell_top')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in rows:
        for j in columns:
                #Define the edges of the mesh at each location
                l_edge = LL[0]+mesh_x*(j-1)
                b_edge = UR[1]-mesh_y*i
                r_edge = LL[0]+mesh_x*j
                t_edge = UR[1]-mesh_y*(i-1)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Takes polygons from the copy layer in a given search region to keep in the "touching" pya.Region 
                search_region = pya.DBox(l_edge,b_edge,r_edge,t_edge)
                touching = pya.Region(top_cell.begin_shapes_rec_touching(copy_lyr, search_region))
                bias_cell_top.shapes(copy_lyr).insert(touching)

                #Biases "touching" and copies it into the bias layer
                touch_up = touching.sized(int(biasing))
                bias_cell_top.shapes(bias_lyr).insert(touch_up)
                

                print(f"Left {l_edge}, bottom {b_edge}, right {r_edge}, top {t_edge}")
                print(f"Finished row {i} and column {j}")
    
    #Merges any overlapping polygons and places the final biased data onto the "final" layer
    bias_merge = pya.Region(bias_cell_top.begin_shapes_rec_touching(bias_lyr,pya.DBox(LL[0],LL[1],UR[0],UR[1])))
    bias_merge.merge()
    bias_cell_top.shapes(final_lyr).insert(bias_merge)

    #return mesh_x, j, i, LL[0], LL[1]
    return gds.write('Test RDP.gds')




rpdb()



16
Left -5.0, bottom 4.375, right -4.333333333333333, top 5.0
Finished row 1 and column 1
Left -4.333333333333333, bottom 4.375, right -3.666666666666667, top 5.0
Finished row 1 and column 2
Left -3.666666666666667, bottom 4.375, right -3.0, top 5.0
Finished row 1 and column 3
Left -3.0, bottom 4.375, right -2.3333333333333335, top 5.0
Finished row 1 and column 4
Left -2.3333333333333335, bottom 4.375, right -1.666666666666667, top 5.0
Finished row 1 and column 5
Left -1.666666666666667, bottom 4.375, right -1.0, top 5.0
Finished row 1 and column 6
Left -1.0, bottom 4.375, right -0.3333333333333339, top 5.0
Finished row 1 and column 7
Left -0.3333333333333339, bottom 4.375, right 0.33333333333333304, top 5.0
Finished row 1 and column 8
Left 0.33333333333333304, bottom 4.375, right 1.0, top 5.0
Finished row 1 and column 9
Left 1.0, bottom 4.375, right 1.666666666666666, top 5.0
Finished row 1 and column 10
Left 1.666666666666666, bottom 4.375, right 2.333333333333333, top 5.0
Finished r